# Parameter Sweep with WaterTAP

The parameter sweep tool is an analysis tool developed by WaterTAP to explore the effect of changing model parameters or decision variables in a WaterTAP model. Using the tool requires a stable WaterTAP model. In this demonstration, we will use the RO with ERD flowsheet developed in a previous session to examing the impact of varying water permeability and water recovery on performance and technoeconomic metrics.

# Imports

We will import our working flowsheet from an existing Python file that contains all the functions necessary to build, set operating conditions, scale, initialize, and solve our model.

In [ ]:
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from IPython.display import clear_output
from pyomo.environ import Objective, value, units as pyunits

from watertap.core.solvers import get_solver

from parameter_sweep import (
    LinearSample,
    parameter_sweep,
)

# Import existing flowsheet module
import RO_with_ERD as ro_erd

solver = get_solver()

In [ ]:
def build_and_solve():

    m = ro_erd.build()
    ro_erd.scale_system(m)
    ro_erd.add_costing(m)
    ro_erd.initialize_system(m)
    _ = ro_erd.solve_system(m)
    clear_output(wait=False)

    return m


m = build_and_solve()
base_lcow = value(m.fs.costing.LCOW)
base_A_comp = value(
    pyunits.convert(
        m.fs.RO.A_comp[0, "H2O"],
        to_units=pyunits.liter / pyunits.m**2 / pyunits.hour / pyunits.bar,
    )
)
base_recovery = value(m.fs.RO.recovery_vol_phase[0, "Liq"])
base_mem_area = value(m.fs.RO.area)
base_pressure = value(
    pyunits.convert(
        m.fs.pump.control_volume.properties_out[0].pressure, to_units=pyunits.bar
    )
)
print(f"\nBase LCOW: {base_lcow:.2f} $/m3")

## Parameter sweep *without* the ParameterSweep tool

This cell is a demonstration of the steps the `ParameterSweep` tool is taking presented in a serial fashion:

1. Define parameter space to be swept
2. Define the results to store
3. Create an initial instance of the model
4. Unfix design variables (if necessary)
5. Fix sweep variables to sweep value
6. Solve model
7. Store results

In [ ]:
# Define number of samples and parameter ranges
num_samples = 10

# Recovery range
recoverys = np.linspace(0.1, 0.75, num_samples)

# Create lists to store results
lcows = list()
pressure = list()

# Create an initial instance of the model
m = build_and_solve()

# Unfix variables to perform the sweep
# Currently water recovery is fixed, but varying recovery will affect design.
# We assume the length of the membrane is fixed to our initial design
# and unfix pump pressure to see how it changes with recovery
m.fs.pump.control_volume.properties_out[0].pressure.unfix()
m.fs.RO.length.fix(value(m.fs.RO.length))

# Perform serial sweep over recovery values
for wr in recoverys:
    m.fs.RO.recovery_vol_phase[0, "Liq"].fix(wr)
    results = ro_erd.solve_system(m)
    lcows.append(value(m.fs.costing.LCOW))
    pressure.append(
        value(
            pyunits.convert(
                m.fs.pump.control_volume.properties_out[0].pressure,
                to_units=pyunits.bar,
            )
        )
    )

# Plot results
fig, ax1 = plt.subplots()
ax1.plot(recoverys, lcows, ls=":", marker=".")
ax1.scatter(
    [base_recovery],
    [base_lcow],
    color="red",
    label="Base Case",
    marker="*",
    edgecolor="k",
    s=200,
)

ax1.set(
    xlabel="Recovery (%)",
    ylabel="Levelized Cost of Water ($/m³)",
)
ax1.set_title("LCOW vs. Recovery")
ax1.legend()

fig, ax2 = plt.subplots()

ax2.plot(recoverys, pressure, ls=":", marker=".")
ax2.scatter(
    [base_recovery],
    [base_pressure],
    color="red",
    label="Base Case",
    marker="*",
    edgecolor="k",
    s=200,
)
ax2.set(
    xlabel="Recovery (%)",
    ylabel="Pressure (bar)",
)
ax2.set_title("Pressure vs. Recovery")
ax2.legend()

# Building a parameter sweep in WaterTAP

A primary advantage to using the parameter sweep tool over the serial sweep demonstration in the previous cell is the ability to run more samples quickly.
Similar our serial sweep, to create a `ParameterSweep` we must provide:

- A function that builds, scales, initializes, and solves an initial flowsheet. Importantly, this function *must return the model*.
- A function that reutrns a dictionary of desired model outputs
- A function that returns the desired sweep parameters

## 1. Model build function

This is the same function we used above to perform our serial sweep above. However, we add lines to fix/unfix the variables necessary to perform our sweep. In this case, we will fix our system design by fixing the membrane length to the initial value and unfix recovery to explore how sweeping membrane permeability will impact performance.

In [ ]:
def build_and_solve():
    """
    Build and solve the RO with ERD model.
    """

    m = ro_erd.build()
    ro_erd.scale_system(m)
    ro_erd.add_costing(m)
    ro_erd.initialize_system(m)
    _ = ro_erd.solve_system(m)
    clear_output(wait=False)

    # Set system design by fixing membrane length
    m.fs.RO.length.fix(value(m.fs.RO.length))
    # Unfix recovery
    m.fs.pump.control_volume.properties_out[0].pressure.unfix()

    return m

## 2. Sweep outputs function

The first positional argument in this function must be our model. We instantiate an empty dictionary and then can add key/value pairs that correspond to the specific model outputs we want. In this example, we are interested in the LCOW, LCOW breakdowns, and SEC.

NOTE: The key must be a string, and the value must be a Pyomo modeling object (e.g., `Var`, `Expression`, etc.)

In [ ]:
def build_outputs(m):
    """
    Create dictionary of outputs to record from the model.
    """
    outputs = {}

    outputs["Total Capital Cost"] = m.fs.costing.total_capital_cost
    outputs["Total Operating Cost"] = m.fs.costing.total_operating_cost
    outputs["LCOW"] = m.fs.costing.LCOW
    outputs["SEC"] = m.fs.costing.SEC

    # Unit-level outputs
    outputs["Pressure"] = m.fs.pump.control_volume.properties_out[0].pressure
    # outputs["Recovery"] = m.fs.RO.recovery_vol_phase[0, "Liq"]
    outputs["Membrane Area"] = m.fs.RO.area
    outputs["Membrane Permeability"] = m.fs.RO.A_comp[0, "H2O"]
    outputs["LCOW Direct CAPEX Pump"] = m.fs.costing.LCOW_component_direct_capex[
        "fs.pump"
    ]
    outputs["LCOW Indirect CAPEX Pump"] = m.fs.costing.LCOW_component_indirect_capex[
        "fs.pump"
    ]
    outputs["LCOW Direct CAPEX ERD"] = m.fs.costing.LCOW_component_direct_capex[
        "fs.erd"
    ]
    outputs["LCOW Indirect CAPEX ERD"] = m.fs.costing.LCOW_component_indirect_capex[
        "fs.erd"
    ]
    outputs["LCOW Direct CAPEX RO"] = m.fs.costing.LCOW_component_direct_capex["fs.RO"]
    outputs["LCOW Indirect CAPEX RO"] = m.fs.costing.LCOW_component_indirect_capex[
        "fs.RO"
    ]
    outputs["LCOW Fixed OPEX Pump"] = m.fs.costing.LCOW_component_fixed_opex["fs.pump"]
    outputs["LCOW Fixed OPEX RO"] = m.fs.costing.LCOW_component_fixed_opex["fs.RO"]
    outputs["LCOW Fixed OPEX ERD"] = m.fs.costing.LCOW_component_fixed_opex["fs.erd"]
    outputs["LCOW Variable OPEX Electricity"] = (
        m.fs.costing.LCOW_aggregate_variable_opex["electricity"]
    )
    return outputs

## 3. Sweep inputs function

Like the sweep outputs function, the first argument must be our model. Then we create a `LinearSample` that defines the variable to be swept and the range of values.

NOTE: Add note on other sampling types?

In [ ]:
def build_sweep_params(m, num_samples=10):

    sweep_params = {}

    # sweep_params["A_comp"] = LinearSample(m.fs.RO.A_comp, 1.0e-12, 6e-12, num_samples)
    sweep_params["Water Recovery"] = LinearSample(
        m.fs.RO.recovery_vol_phase, 0.1, 0.75, num_samples
    )

    return sweep_params

## 4. Run the parameter sweep

We can now pass these different functions to the `parameter_sweep` function to run. 

In [ ]:
num_samples = 10
file_save = "parameter_sweep_results.csv"

results_array, results_dict = parameter_sweep(
    build_model=build_and_solve,
    build_sweep_params=build_sweep_params,
    build_sweep_params_kwargs={"num_samples": num_samples},
    build_outputs=build_outputs,
    num_samples=num_samples,
    csv_results_file_name=file_save,
)

# Plotting

In [ ]:
def make_stacked_plot(
    file_save="parameter_sweep_results.csv",
    parameter="Water Recovery",
    units=["RO", "ERD", "Pump"],
    unit_colors=["#1f77b4", "#ff7f0e", "#2ca02c"],
    elec_color="#db2020ab",
    opex_hatch="//",
    capex_hatch="..",
    elec_hatch="++",
    fontsize=14,
):
    
    """
    Create stacked plot of LCOW contributions from single parameter sweep results.
    """

    ################################################

    df = pd.read_csv(file_save)
    df.set_index(f"# {parameter}", inplace=True)
    df.sort_values(by=f"# {parameter}", inplace=True)

    unit_color_dict = dict(zip(units, unit_colors))

    capex_lcow = defaultdict(list)
    opex_lcow = defaultdict(list)
    agg_flow_lcow = defaultdict(list)

    capex_lcow_rel = defaultdict(list)
    opex_lcow_rel = defaultdict(list)
    agg_flow_lcow_rel = defaultdict(list)

    for _, row in df.iterrows():
        lcow = row["LCOW"]
        for u in units:
            c = row[f"LCOW Direct CAPEX {u}"] + row[f"LCOW Indirect CAPEX {u}"]
            o = row[f"LCOW Fixed OPEX {u}"]
            capex_lcow[u].append(c)
            opex_lcow[u].append(o)

            capex_lcow_rel[u].append(c / lcow)
            opex_lcow_rel[u].append(o / lcow)

        agg_flow_lcow["electricity"].append(row["LCOW Variable OPEX Electricity"])
        agg_flow_lcow_rel["electricity"].append(
            row["LCOW Variable OPEX Electricity"] / lcow
        )

    ################################################
    # Collecting data for stacked plot

    stacked_cols = list()
    stacked_labels = list()
    stacked_hatch = list()
    stacked_colors = list()

    stacked_cols_rel = list()
    stacked_labels_rel = list()
    stacked_hatch_rel = list()
    stacked_colors_rel = list()

    stacked_cols.append(agg_flow_lcow["electricity"])
    stacked_labels.append("Electricity")
    stacked_hatch.append(elec_hatch)
    stacked_colors.append(elec_color)

    stacked_cols_rel.append(agg_flow_lcow_rel["electricity"])
    stacked_labels_rel.append("Electricity")
    stacked_hatch_rel.append(elec_hatch)
    stacked_colors_rel.append(elec_color)

    for u in units:
        stacked_cols.append(opex_lcow[u])
        stacked_colors.append(unit_color_dict[u])
        stacked_hatch.append(opex_hatch)

        stacked_cols.append(capex_lcow[u])
        stacked_colors.append(unit_color_dict[u])
        stacked_hatch.append(capex_hatch)

        stacked_cols_rel.append(opex_lcow_rel[u])
        stacked_colors_rel.append(unit_color_dict[u])
        stacked_hatch_rel.append(opex_hatch)

        stacked_cols_rel.append(capex_lcow_rel[u])
        stacked_colors_rel.append(unit_color_dict[u])
        stacked_hatch_rel.append(capex_hatch)

    ################################################
    # Plotting absolute LCOW contributions

    fig, ax = plt.subplots()

    ax.stackplot(
        df.index,
        stacked_cols,
        colors=stacked_colors,
        # labels=stacked_labels,
        edgecolor="black",
        hatch=stacked_hatch,
    )

    legend_elements = [
        Patch(facecolor="white", hatch=capex_hatch, label="CAPEX", edgecolor="k"),
        Patch(facecolor="white", hatch=opex_hatch, label="OPEX", edgecolor="k"),
        Patch(
            facecolor=elec_color,
            label="Electricity",
            hatch=elec_hatch,
            edgecolor="k",
        ),
    ] + [Patch(facecolor=unit_color_dict[u], label=u, edgecolor="k") for u in units]

    leg_kwargs = dict(
        loc="lower left",
        frameon=False,
        ncol=4,
        handlelength=1,
        handleheight=1,
        labelspacing=0.2,
        columnspacing=0.9,
        bbox_to_anchor=(0.0, 1.02, 1.0, 0.102),
        mode="expand",
    )
    ax.legend(handles=legend_elements, **leg_kwargs)
    # ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{int(x*100)}%"))
    ax.set_xlabel(f"{parameter}", fontsize=fontsize)
    ax.set_ylabel("LCOW ($/m³)", fontsize=fontsize)
    ax.tick_params(axis="both", labelsize=fontsize)

    ################################################
    # Plotting relative LCOW contributions
    fig, ax = plt.subplots()

    ax.stackplot(
        df.index,
        stacked_cols_rel,
        colors=stacked_colors_rel,
        # labels=stacked_labels_rel,
        edgecolor="black",
        hatch=stacked_hatch_rel,
    )

    legend_elements = [
        Patch(facecolor="white", hatch=capex_hatch, label="CAPEX", edgecolor="k"),
        Patch(facecolor="white", hatch=opex_hatch, label="OPEX", edgecolor="k"),
        Patch(
            facecolor=elec_color,
            label="Electricity",
            hatch=elec_hatch,
            edgecolor="k",
        ),
    ] + [Patch(facecolor=unit_color_dict[u], label=u, edgecolor="k") for u in units]

    ax.legend(handles=legend_elements, **leg_kwargs)
    # ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{int(x*100)}%"))
    ax.set_xlabel(f"{parameter}", fontsize=fontsize)
    ax.set_ylabel("Relative LCOW", fontsize=fontsize)
    ax.tick_params(axis="both", labelsize=fontsize)


# make_stacked_plot(file_save)

# Two Parameter Sweep

Unfix Pressure

Unfix Area

Unfix Recovery

In [ ]:
def build_and_solve():
    """
    Build and solve the RO with ERD model.
    """

    m = ro_erd.build()
    ro_erd.scale_system(m)
    ro_erd.add_costing(m)
    ro_erd.initialize_system(m)
    _ = ro_erd.solve_system(m)
    clear_output(wait=False)

    m.fs.pump.control_volume.properties_out[0].pressure.unfix()
    # m.fs.RO.recovery_vol_phase[0, "Liq"].fix(0.5)
    m.fs.RO.recovery_vol_phase.unfix()
    m.fs.RO.area.unfix()

    m.fs.obj = Objective(expr=m.fs.costing.LCOW)

    return m


def build_outputs(m):
    """
    Create dictionary of outputs to record from the model.
    """
    outputs = {}

    outputs["Total Capital Cost"] = m.fs.costing.total_capital_cost
    outputs["Total Operating Cost"] = m.fs.costing.total_operating_cost
    outputs["LCOW"] = m.fs.costing.LCOW
    outputs["SEC"] = m.fs.costing.SEC

    # Unit-level outputs
    outputs["Pressure"] = m.fs.pump.control_volume.properties_out[0].pressure
    outputs["Recovery"] = m.fs.RO.recovery_vol_phase[0, "Liq"]
    outputs["Membrane Area"] = m.fs.RO.area
    outputs["Membrane Permeability"] = m.fs.RO.A_comp[0, "H2O"]
    outputs["LCOW Direct CAPEX Pump"] = m.fs.costing.LCOW_component_direct_capex[
        "fs.pump"
    ]
    outputs["LCOW Indirect CAPEX Pump"] = m.fs.costing.LCOW_component_indirect_capex[
        "fs.pump"
    ]
    outputs["LCOW Direct CAPEX ERD"] = m.fs.costing.LCOW_component_direct_capex[
        "fs.erd"
    ]
    outputs["LCOW Indirect CAPEX ERD"] = m.fs.costing.LCOW_component_indirect_capex[
        "fs.erd"
    ]
    outputs["LCOW Direct CAPEX RO"] = m.fs.costing.LCOW_component_direct_capex["fs.RO"]
    outputs["LCOW Indirect CAPEX RO"] = m.fs.costing.LCOW_component_indirect_capex[
        "fs.RO"
    ]
    outputs["LCOW Fixed OPEX Pump"] = m.fs.costing.LCOW_component_fixed_opex["fs.pump"]
    outputs["LCOW Fixed OPEX RO"] = m.fs.costing.LCOW_component_fixed_opex["fs.RO"]
    outputs["LCOW Fixed OPEX ERD"] = m.fs.costing.LCOW_component_fixed_opex["fs.erd"]
    outputs["LCOW Variable OPEX Electricity"] = (
        m.fs.costing.LCOW_aggregate_variable_opex["electricity"]
    )
    return outputs


def build_sweep_params(m, num_samples=10):

    sweep_params = {}

    # sweep_params["A_comp"] = LinearSample(m.fs.RO.A_comp, 1.0e-12, 6e-12, num_samples)
    sweep_params["Pressure"] = LinearSample(
        m.fs.pump.control_volume.properties_out[0].pressure, 60e5, 100e5, num_samples
    )
    sweep_params["Area"] = LinearSample(m.fs.RO.area, 25, 85, num_samples)

    return sweep_params


num_samples = 25
file_save = "parameter_sweep_results.csv"

results_array, results_dict = parameter_sweep(
    build_model=build_and_solve,
    build_sweep_params=build_sweep_params,
    build_sweep_params_kwargs={"num_samples": num_samples},
    build_outputs=build_outputs,
    num_samples=num_samples,
    csv_results_file_name=file_save,
)

In [ ]:
from scipy.interpolate import griddata


def plot_contour(
    df,
    x="fs.flow_mgd",
    y="tds",
    z="fs.costing.LCOW",
    min_x=None,
    min_y=None,
    max_x=None,
    max_y=None,
    x_adj=1,
    y_adj=1,
    z_adj=1,
    approach="interpolate",
    cmap="turbo",
    interp_method="cubic",
    grid_len=100,
    levels=20,
    levelsf=None,
    set_dict=dict(),
    cb_title="",
    cb_fontsize=14,
    contour_label_fmt="  %#.2e  ",
    add_contour_labels=False,
    fig=None,
    ax=None,
    figsize=(6, 4),
):
    """
    Generic function to create contour plot from
    three columns in DataFrame
    """
    # if (fig, ax) == (None, None):
    fig, ax = plt.subplots(figsize=figsize)
    fig.set_size_inches(figsize[0], figsize[1], forward=True)

    if min_x is not None:
        df = df[df[x] >= min_x].copy()
    if min_y is not None:
        df = df[df[y] >= min_y].copy()

    if max_x is not None:
        df = df[df[x] <= min_x].copy()
    if max_y is not None:
        df = df[df[y] <= min_y].copy()

    if levelsf is None:
        levelsf = levels

    if z_adj is not None:
        df[z] = df[z] * z_adj

    if approach == "pivot":
        pivot = df.pivot(index=y, columns=x, values=z * z_adj)

        x = pivot.columns.values
        y = pivot.index.values
        z = pivot.values

        contourf = ax.contourf(x, y, z, levels=levels, cmap=cmap)
        cb = plt.colorbar(contourf, label=cb_title)

    if approach == "interpolate":

        x = df[x] * x_adj
        y = df[y] * y_adj
        z = df[z] * z_adj

        xi = np.linspace(min(x), max(x), grid_len)
        yi = np.linspace(min(y), max(y), grid_len)
        xi, yi = np.meshgrid(xi, yi)

        zi = griddata((x, y), z, (xi, yi), method=interp_method)

        contourf = ax.contourf(xi, yi, zi, levels=levels, cmap=cmap)
        cb = plt.colorbar(contourf, label=cb_title)

    ax.set(**set_dict)

    if add_contour_labels:
        contour = ax.contour(
            contourf,
            levelsf,
            colors="k",
            linestyles="dashed",
        )
        ax.clabel(contour, colors="black", fmt=contour_label_fmt, fontsize=cb_fontsize)

    fig.tight_layout()

    return fig, ax


df = pd.read_csv(file_save)
fig, ax = plot_contour(
    df,
    x="# Pressure",
    y="Area",
    z="LCOW",
    x_adj=1e-5,
    y_adj=1,
    z_adj=1,
    cb_title="LCOW ($/m³)",
    set_dict={
        "xlabel": "Pressure (bar)",
        "ylabel": "Membrane Area (m²)",
        "title": "LCOW Contour Plot",
    },
    # add_contour_labels=True,
    # contour_label_fmt="  %#.2f  ",
    # levels=5,
    # cb_fontsize=14,
)
ax.get_xlim()
ax.plot([base_pressure], [base_mem_area], marker="*", color="red", markersize=15)

fig, ax = plot_contour(
    df,
    x="# Pressure",
    y="Area",
    z="Recovery",
    x_adj=1e-5,
    y_adj=1,
    z_adj=1,
    cb_title="LCOW ($/m³)",
    cmap="viridis",
    set_dict={
        "xlabel": "Pressure (bar)",
        "ylabel": "Membrane Area (m²)",
        "title": "Water Recovery Contour Plot",
    },
    # add_contour_labels=True,
)
ax.get_xlim()
ax.plot([base_pressure], [base_mem_area], marker="*", color="red", markersize=15)

# Try It Yourself
We want to do a single parameter sweep for membrane permeability

- Create the `build_and_solve` function. The function should fix the membrane length and unfix the RO recovery.
- Create the `build_sweep_params` function and designate membrane permeability `m.fs.RO.A_comp` as a `LinearSample` from 1e-6 to 6e-6 m/s/Pa. This function should be able to accept a `number_samples` keyword argument.
- You can use the `build_outputs` function from previous examples in this tutorial.
- Run the parameter sweep for 50 samples.

In [ ]:
def build_and_solve():
    """
    Build and solve the RO with ERD model.
    """

    m = ro_erd.build()
    ro_erd.scale_system(m)
    ro_erd.add_costing(m)
    ro_erd.initialize_system(m)
    _ = ro_erd.solve_system(m)
    clear_output(wait=False)

    # Set system design by fixing membrane length
    m.fs.RO.length.fix(value(m.fs.RO.length))
    # Unfix recovery
    m.fs.RO.recovery_vol_phase.unfix()

    return m


def build_outputs(m):
    """
    Create dictionary of outputs to record from the model.
    """
    outputs = {}

    outputs["Total Capital Cost"] = m.fs.costing.total_capital_cost
    outputs["Total Operating Cost"] = m.fs.costing.total_operating_cost
    outputs["LCOW"] = m.fs.costing.LCOW
    outputs["SEC"] = m.fs.costing.SEC

    # Unit-level outputs
    outputs["Pressure"] = m.fs.pump.control_volume.properties_out[0].pressure
    outputs["Recovery"] = m.fs.RO.recovery_vol_phase[0, "Liq"]
    outputs["Membrane Area"] = m.fs.RO.area
    outputs["Membrane Permeability"] = m.fs.RO.A_comp[0, "H2O"]
    outputs["LCOW Direct CAPEX Pump"] = m.fs.costing.LCOW_component_direct_capex[
        "fs.pump"
    ]
    outputs["LCOW Indirect CAPEX Pump"] = m.fs.costing.LCOW_component_indirect_capex[
        "fs.pump"
    ]
    outputs["LCOW Direct CAPEX ERD"] = m.fs.costing.LCOW_component_direct_capex[
        "fs.erd"
    ]
    outputs["LCOW Indirect CAPEX ERD"] = m.fs.costing.LCOW_component_indirect_capex[
        "fs.erd"
    ]
    outputs["LCOW Direct CAPEX RO"] = m.fs.costing.LCOW_component_direct_capex["fs.RO"]
    outputs["LCOW Indirect CAPEX RO"] = m.fs.costing.LCOW_component_indirect_capex[
        "fs.RO"
    ]
    outputs["LCOW Fixed OPEX Pump"] = m.fs.costing.LCOW_component_fixed_opex["fs.pump"]
    outputs["LCOW Fixed OPEX RO"] = m.fs.costing.LCOW_component_fixed_opex["fs.RO"]
    outputs["LCOW Fixed OPEX ERD"] = m.fs.costing.LCOW_component_fixed_opex["fs.erd"]
    outputs["LCOW Variable OPEX Electricity"] = (
        m.fs.costing.LCOW_aggregate_variable_opex["electricity"]
    )
    return outputs


def build_sweep_params(m, num_samples=10):

    sweep_params = {}

    sweep_params["A_comp"] = LinearSample(m.fs.RO.A_comp, 1.0e-12, 6e-12, num_samples)
    return sweep_params


num_samples = 100
file_save = "parameter_sweep_results.csv"

# results_array, results_dict = parameter_sweep(
#     build_model=build_and_solve,
#     build_sweep_params=build_sweep_params,
#     build_sweep_params_kwargs={"num_samples": num_samples},
#     build_outputs=build_outputs,
#     num_samples=num_samples,
#     csv_results_file_name=file_save,
# )

df = pd.read_csv(file_save)
make_stacked_plot(file_save, parameter="A_comp")